In [ ]:
!pip install -q --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu121
!pip install -q openai-whisper
!pip install -q yt-dlp
import os
import subprocess
import sys
import logging
from google.colab import drive
from google.colab import files
from yt_dlp import YoutubeDL
from yt_dlp.utils import DownloadError as YoutubeDLDownloadError

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def check_dependencies():
    """Checks if ffmpeg, ffprobe, and yt-dlp are installed."""
    logging.info("Checking for required dependencies...")
    dependencies = {
        "ffmpeg": "sudo apt-get update && sudo apt-get install -y ffmpeg",
        "ffprobe": "sudo apt-get update && sudo apt-get install -y ffmpeg",
        "yt-dlp": "pip install yt-dlp"
    }

    for dep, install_cmd in dependencies.items():
        try:
            if dep == "yt-dlp":
                import yt_dlp
                logging.info(f"✅ {dep} found.")
            else:
                subprocess.run([dep, "-version"], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                logging.info(f"✅ {dep} found.")
        except (subprocess.CalledProcessError, FileNotFoundError, ImportError):
            logging.error(f"❌ {dep} not found. Please install it using: {install_cmd}")
            if "apt-get" in install_cmd:
                logging.info(f"Attempting to install {dep} via apt-get...")
                try:
                    subprocess.run(install_cmd.split(), check=True)
                    logging.info(f"✅ {dep} installed successfully.")
                except subprocess.CalledProcessError as e:
                    logging.error(f"Failed to install {dep}: {e}")
                    sys.exit(1)
            elif "pip install" in install_cmd:
                logging.info(f"Attempting to install {dep} via pip...")
                try:
                    subprocess.run([sys.executable, "-m", "pip", "install", "yt-dlp"], check=True)
                    logging.info(f"✅ {dep} installed successfully.")
                except subprocess.CalledProcessError as e:
                    logging.error(f"Failed to install {dep}: {e}")
                    sys.exit(1)
            else:
                sys.exit(1)

def mount_drive():
    """Mounts Google Drive."""
    try:
        drive.mount('/content/drive')
        logging.info("✅ Google Drive mounted successfully.")
    except Exception as e:
        logging.error(f"❌ Failed to mount Google Drive: {e}")
        sys.exit(1)

def upload_cookies():
    """Uploads cookies.txt file and returns its path."""
    logging.info("📤 Please upload your YouTube cookies.txt file (use the browser extension 'Get cookies.txt'). This is optional, but recommended for age-restricted or private videos.")
    try:
        uploaded = files.upload()
        if not uploaded:
            logging.warning("⚠️ No file uploaded. Proceeding without cookies.")
            return None

        cookies_file_name = list(uploaded.keys())[0]
        cookies_path = os.path.join("/content", cookies_file_name)

        logging.info(f"✅ Cookies file '{cookies_file_name}' uploaded to {cookies_path}")
        return cookies_path
    except Exception as e:
        logging.error(f"❌ Error during cookies file upload: {e}")
        return None

def download_video_to_drive(url, drive_folder_name="Downloaded_YouTube_Videos", cookies_path=None):
    """
    Downloads a YouTube video to a specified Google Drive folder using yt-dlp,
    with optional cookies for authentication.
    The filename will be based on the video's title.
    """
    base_drive_path = "/content/drive/MyDrive"
    output_directory = os.path.join(base_drive_path, drive_folder_name)
    os.makedirs(output_directory, exist_ok=True)
    logging.info(f"📥 Attempting to download video from: {url}")
    logging.info(f"📁 Videos will be saved to: {output_directory}")

    try:
        with YoutubeDL({'quiet': True, 'simulate': True, 'force_generic_extractor': True}) as ydl:
            info_dict = ydl.extract_info(url, download=False)
            video_title = info_dict.get('title', 'video').replace('/', '_').replace('\\', '_')
            if len(video_title) > 100:
                video_title = video_title[:100] + "..."
            logging.info(f"Identified video title: '{video_title}'")
    except Exception as e:
        logging.warning(f"Could not extract video title, using generic filename. Error: {e}")
        video_title = "downloaded_video"

    output_template = os.path.join(output_directory, f"{video_title}.%(ext)s")

    ydl_opts = {
        "format": "bestvideo[height<=720]+bestaudio/best[height<=720]/best",
        "outtmpl": output_template,
        "merge_output_format": "mp4",
        "progress": True,
        "noplaylist": True,
    }

    if cookies_path and os.path.exists(cookies_path):
        ydl_opts["cookiefile"] = cookies_path
        logging.info("Using provided cookies for download.")
    else:
        logging.warning("No valid cookies file provided or found. Download might fail for age-restricted/private videos.")

    try:
        with YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        final_output_path = ydl.prepare_filename(ydl_opts)
        logging.info(f"✅ Download complete: {final_output_path}")
        return final_output_path
    except YoutubeDLDownloadError as e:
        logging.error(f"❌ yt-dlp download failed for '{url}': {e}")
        return None
    except Exception as e:
        logging.error(f"❌ An unexpected error occurred during video download: {e}")
        return None

def main():
    logging.info("--- Starting Video Download Pipeline ---")

    check_dependencies()
    mount_drive()

    cookies_path = upload_cookies()

    while True:
        video_url = input("\nPlease enter the YouTube video URL (or type 'done' to exit): ").strip()
        if video_url.lower() == 'done':
            logging.info("Exiting video download process.")
            break
        elif not video_url:
            logging.warning("⚠️ No video URL provided. Please enter a valid URL or 'done'.")
            continue

        downloaded_path = download_video_to_drive(video_url, cookies_path=cookies_path)

        if downloaded_path:
            logging.info(f"Successfully downloaded '{video_url}' to: {downloaded_path}")
        else:
            logging.error(f"❌ Failed to download video from: {video_url}")

    if cookies_path and os.path.exists(cookies_path):
        try:
            os.remove(cookies_path)
            logging.info(f"🗑️ Cleaned up temporary cookies file: {cookies_path}")
        except Exception as e:
            logging.warning(f"⚠️ Failed to remove cookies file {cookies_path}: {e}")

    logging.info("--- Pipeline Finished ---")

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


KeyboardInterrupt: 